# Advanced Topics in Stochastic Modelling - Mini Project
- Chirag Shivakumar 1004996
- Tze Liang 100xxxx

# Project Name - Use of regenerative simulation method for staffing call centers

## Project Objective 
Minimize the number of call center agents while ensuring service quality metrics are met 
- less than 2% abandonment and 
- average wait time under five seconds


The project involves using a regenerative simulation method to determine the optimal staffing level in a call center to meet specified performance constraints with 95% confidence.

The project involves using a regenerative simulation method to determine the optimal staffing level in a call center to meet specified performance constraints with 95% confidence. Here is a detailed breakdown of the project parameters and requirements:

### Motivation
- Call centers have seen a surge in growth and spend over $300 billion worldwide.
- The challenge lies in balancing operational efficiency and service quality.
- Personnel costs make up about 70% of a call center’s expenditure.

### Call Center Staffing Aspects
- The right number of agents is crucial to avoid understaffing, which can lead to poor service and customer frustration.
- A study by Purdue University indicates that 63% of customers cite a negative call center experience as a reason for stopping transactions with a company.
- The model must consider customer abandonment.

### Call Center Model
- A moderate-sized call center with an average of 20 arrivals per minute over a given time interval is considered.
- The average service time per call is three minutes ($\mu = \frac{1}{3}$).
- Arrivals are according to a Poisson process, and service times are exponentially distributed.
- The call center operates on a first-come-first-serve (FCFS) basis.
- Each inbound caller has a generally distributed patience time $ T $. If the wait time exceeds $ T $, the caller abandons the call; otherwise, they get served.

### Target Quality of Service
- The probability of abandonment should be less than 2%.
- The average wait should be less than five seconds.
- It is advised to choose more than $\lambda / \mu = 60$ agents to meet these constraints, but the exact number needed is unclear.

### Task Description
- Use regenerative simulation output analysis to identify the minimum number of call center agents needed to meet the target performance constraint with 95% confidence.
- Assume customer patience times are independently and identically distributed (i.i.d.) uniformly over the interval [0,6] minutes.
- Determine the minimum staffing levels that meet both target performance constraints i) and ii) with 95% confidence.

In summary, the project's aim is to apply simulation techniques to find the least number of call center agents required to ensure that the probability of a customer abandoning a call is below 2%, and the average waiting time is under five seconds. This will involve modeling the call center's traffic as a Poisson process with exponentially distributed service times and uniformly distributed patience times, and then simulating different staffing levels to find the optimal number of agents that satisfies the service level constraints with high confidence.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import expon, uniform, norm

# Call Center Simulation Constants

This document outlines the constants used in the call center simulation.

## Simulation Parameters

- `HOURS_OPEN`: The total hours the call center is open. For this simulation, we set it to 8 hours.
- `SECONDS_OPEN`: Converts the `HOURS_OPEN` into seconds for the simulation.
- `ARRIVALS_PER_MINUTE`: The average number of call arrivals per minute, set to 20.
- `MEAN_SERVICE_TIME`: The average service time per call in seconds (3 minutes represented in seconds).

## Rate Constants

- `LAMBDA`: The arrival rate per second calculated from `ARRIVALS_PER_MINUTE`.
- `MU`: The service rate per second, which is the inverse of the `MEAN_SERVICE_TIME`.

## Quality of Service Targets

- `MAX_WAIT_TIME`: The maximum average wait time allowed in the simulation (5 seconds).
- `MAX_ABANDONMENT_RATE`: The maximum rate at which calls can be abandoned (2%).

## Patience Time Distribution

- `MIN_PATIENCE_TIME`: The minimum patience time of a caller in seconds.
- `MAX_PATIENCE_TIME`: The maximum patience time of a caller in seconds (6 minutes represented in seconds).

In [ ]:
HOURS_OPEN = 8  
SECONDS_OPEN = HOURS_OPEN * 3600  
ARRIVALS_PER_MINUTE = 20  
MEAN_SERVICE_TIME = 3 * 60  

LAMBDA = ARRIVALS_PER_MINUTE / 60  
MU = 1 / MEAN_SERVICE_TIME  

MAX_WAIT_TIME = 5  
MAX_ABANDONMENT_RATE = 0.02  

MIN_PATIENCE_TIME = 0  
MAX_PATIENCE_TIME = 6 * 60  

# Visualization on Call Center Volume (Assuming every call is answered instantly)

This visualization represents the call volume at a call center, assuming that every call is answered immediately. The plot displays the number of people on call at each second throughout the call center's operating hours.

## Assumption

- Every call is instantly answered.
- The number of people on a call at any given second equals the number of arrivals up to that second minus those whose calls have been completed.

## Data Preparation

1. **Arrival Times Generation**:
   - Calculated the total number of expected arrivals based on the average arrivals per minute and operating hours.
   - Generated arrival times using an exponential distribution, representing a Poisson arrival process.
   - Cumulative sums of arrival times were used to simulate sequential call arrivals within the operating hours.

2. **Service Times**:
   - Service times for each call were generated using an exponential distribution, with an average service time as specified.

3. **End Times Calculation**:
   - For each call, the end time was calculated as the arrival time plus the service time.

4. **Data Aggregation**:
   - A DataFrame was created with the arrival and end times for each call.

5. **Time Series Analysis**:
   - Generated a second-by-second time series for the entire operating period.
   - Counted the number of ongoing calls at each second.

In [ ]:
total_arrivals = int(ARRIVALS_PER_MINUTE * 60 * HOURS_OPEN)  
arrival_times = np.random.exponential(1 / LAMBDA, size=total_arrivals)
arrival_times = np.cumsum(arrival_times)
arrival_times = arrival_times[arrival_times <= SECONDS_OPEN]  

service_times = np.random.exponential(MEAN_SERVICE_TIME, size=len(arrival_times))

end_times = arrival_times + service_times

In [ ]:
calls_df = pd.DataFrame({
    'arrival_time': arrival_times,
    'end_time': end_times
})
# calls_df

In [ ]:
time_series = np.arange(0, SECONDS_OPEN + 1)

people_on_call = np.array([
    ((calls_df['arrival_time'] <= time) & (calls_df['end_time'] > time)).sum()
    for time in time_series
])


## Visualization 1- Number of People on Call Over Time (Assume: Instant Call Answering)

- A line graph was plotted to represent the number of people on call at every second.
- The x-axis represents time in seconds.
- The y-axis shows the number of people on call.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time_series,
    y=people_on_call,
    mode='lines',
    name='People on Call'
))

fig.update_layout(
    title='Number of People on Call Over Time (Assume: Instant Call Answering)',
    xaxis_title='Time (seconds)',
    yaxis_title='Number of People on Call',
    xaxis=dict(range=[0, SECONDS_OPEN]),  
    yaxis=dict(range=[0, people_on_call.max() + 1])  # max number of people
)

fig.show()

## Visualization 2- Statistical Normal Distribution Fit to Call Volume Data

This visualization demonstrates a normal distribution fit to the call volume data, represented by the number of people on call at each second in a call center.

### Normal Distribution Fitting

- The data representing the number of people on call at each second (`people_on_call`) was fitted to a normal distribution.
- The mean (`mu`) and standard deviation (`std`) of the distribution were calculated using the `norm.fit` method from the `scipy.stats` library.

### Probability Density Function Calculation

- A range of x values, spanning from the minimum to the maximum number of people on call, was created.
- The probability density function (PDF) of the normal distribution was calculated for these x values.

### Plotting

- A histogram was plotted to represent the actual distribution of the call volume data.
    - The number of bins was set to 60 for a detailed view.
    - Histogram bars were colored light blue and set to 75% opacity.
- A red line plot was added to show the PDF of the fitted normal distribution.

### Plot Details

- **Title**: 'Normal Distribution Fit to Number of People on Call'
- **X-axis**: Represents the number of people on call.
- **Y-axis**: Shows the probability density.

In [ ]:
mu, std = norm.fit(people_on_call)

x_values = np.linspace(people_on_call.min(), people_on_call.max(), 100)

pdf = norm.pdf(x_values, mu, std)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=people_on_call,
    histnorm='probability density',
    name='Call Distribution',
    nbinsx=60,  # change accordingly
    marker_color='lightblue',
    opacity=0.75
))

fig.add_trace(go.Scatter(
    x=x_values,
    y=pdf,
    name='Normal Distribution',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Normal Distribution Fit to Number of People on Call',
    xaxis_title='Number of People on Call',
    yaxis_title='Probability Density',
    bargap=0.2,  
)

fig.show()

## Statistical Analysis

- The average number of calls in progress at any given second.
- The peak (maximum) number of calls in progress at any given second.
- The distribution of calls across time to identify peak hours.

In [ ]:
average_calls_in_progress = people_on_call.mean()
print(average_calls_in_progress)

peak_max = people_on_call.max()
print(peak_max)

percentiles = np.percentile(people_on_call, [25, 50, 75, 90, 95])
print(percentiles)

# Part a
# Experiment lo

In [ ]:
# Define a function to run the simulation for a given number of agents
def run_simulation(num_agents, seed=None):
    np.random.seed(seed)
    # Initialize state variables
    wait_times = []
    abandonments = 0
    agents_busy = np.zeros(num_agents)

    # Simulate arrivals
    arrival_times = np.random.exponential(1 / LAMBDA, size=int(ARRIVALS_PER_MINUTE * HOURS_OPEN * 60))
    arrival_times = np.cumsum(arrival_times)
    arrival_times = arrival_times[arrival_times <= SECONDS_OPEN]  # Only consider arrivals within operating hours

    # Simulate service and patience times
    service_times = np.random.exponential(MEAN_SERVICE_TIME, size=len(arrival_times))
    patience_times = uniform.rvs(MIN_PATIENCE_TIME, MAX_PATIENCE_TIME, size=len(arrival_times))

    # Process each arrival
    for arrival_time, service_time, patience_time in zip(arrival_times, service_times, patience_times):
        # Check if any agent is available or will become available before the caller's patience runs out
        available_agent = None
        for i, busy_until in enumerate(agents_busy):
            if busy_until <= arrival_time:
                available_agent = i
                break
            elif busy_until <= arrival_time + patience_time:
                if available_agent is None or busy_until < agents_busy[available_agent]:
                    available_agent = i

        # If no agent is available and won't be before patience runs out, the call is abandoned
        if available_agent is None:
            abandonments += 1
        else:
            wait_time = max(0, agents_busy[available_agent] - arrival_time)
            wait_times.append(wait_time)
            # Update the agent's busy time
            agents_busy[available_agent] = max(arrival_time, agents_busy[available_agent]) + service_time

    # Calculate performance metrics
    average_wait_time = np.mean(wait_times) if wait_times else 0
    abandonment_rate = abandonments / len(arrival_times)

    return {
        'num_agents': num_agents,
        'average_wait_time': average_wait_time,
        'abandonment_rate': abandonment_rate,
        'abandonments': abandonments,
        'total_calls': len(arrival_times),
        'wait_times': wait_times
    }

In [ ]:
# We'll start with the advised number of agents and increase until we meet performance criteria
results = []
num_agents = 58  # Starting point
while True:
    result = run_simulation(num_agents)
    results.append(result)
    # Check if both performance constraints are met
    if result['abandonment_rate'] <= MAX_ABANDONMENT_RATE and result['average_wait_time'] <= MAX_WAIT_TIME:
        break  # Found the optimal number of agents
    num_agents += 1  # Increase number of agents for next simulation run

In [ ]:
# Convert results to a DataFrame for easier analysis and plotting
results_df = pd.DataFrame(results)
results_df  # Display the first few rows to check our simulation results

In [ ]:
fig = go.Figure()

# Add a scatter plot for average wait times
fig.add_trace(go.Scatter(
    x=results_df['num_agents'], 
    y=results_df['average_wait_time'], 
    mode='lines+markers', 
    name='Average Wait Time (seconds)'
))

# Add a scatter plot for abandonment rates
fig.add_trace(go.Scatter(
    x=results_df['num_agents'], 
    y=results_df['abandonment_rate'], 
    mode='lines+markers', 
    name='Abandonment Rate',
    yaxis='y2'  # Assign to the secondary y-axis
))

# Edit the layout
fig.update_layout(
    title='Average Wait Time and Abandonment Rate vs. Number of Agents',
    xaxis_title='Number of Agents',
    yaxis_title='Average Wait Time (seconds)',
    yaxis2=dict(
        title='Abandonment Rate',
        overlaying='y',
        side='right',
        tickformat='.1%'
    ),
    legend=dict(
        x=0.7,
        y=0.95,
        traceorder='reversed',
        title_font_family='Arial',
        font=dict(
            family='Arial',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

# Show plot
fig.show()

In [ ]:
import numpy as np
from scipy.stats import uniform

# Constants from previous context
HOURS_OPEN = 8
SECONDS_OPEN = HOURS_OPEN * 3600
ARRIVALS_PER_MINUTE = 20
MEAN_SERVICE_TIME = 3 * 60
LAMBDA = ARRIVALS_PER_MINUTE / 60
MU = 1 / MEAN_SERVICE_TIME
MAX_WAIT_TIME = 5
MAX_ABANDONMENT_RATE = 0.02
MIN_PATIENCE_TIME = 0
MAX_PATIENCE_TIME = 6 * 60

# Assuming that the target performance constraints are:
# 1) Probability of abandonment should be less than 2%
# 2) Average wait should be less than five seconds

# Define the function to run the call center simulation
def simulate_call_center(num_agents, num_calls, seed=42):
    np.random.seed(seed)
    # Generate call arrivals
    arrival_times = np.cumsum(np.random.exponential(scale=1/LAMBDA, size=num_calls))
    # Service times and patience times
    service_times = np.random.exponential(scale=MEAN_SERVICE_TIME, size=num_calls)
    patience_times = uniform(loc=MIN_PATIENCE_TIME, scale=MAX_PATIENCE_TIME).rvs(size=num_calls)
    
    # Initialize agents to be ready at time 0
    agents = np.zeros(num_agents)
    wait_times = []
    abandonments = 0
    
    for i in range(num_calls):
        if arrival_times[i] > SECONDS_OPEN:  # If the call arrives after closing, do not process
            break
        # Check for the first available agent
        available_agent = np.where(agents <= arrival_times[i])[0]
        if available_agent.size > 0:
            agent_index = available_agent[0]
            agents[agent_index] = arrival_times[i] + service_times[i]  # Update agent's next available time
            wait_times.append(0)  # No wait time since the agent is immediately available
        else:
            # All agents are busy, check if any call will be abandoned
            next_available_time = np.min(agents)
            wait_time = next_available_time - arrival_times[i]
            if wait_time > patience_times[i]:
                abandonments += 1
            else:
                # Wait for the next agent to be available
                agent_index = np.argmin(agents)
                agents[agent_index] += service_times[i]  # Update agent's next available time
                wait_times.append(wait_time)
    
    average_wait_time = np.mean(wait_times) if wait_times else 0
    abandonment_rate = abandonments / num_calls
    
    return average_wait_time, abandonment_rate

# Find the minimum number of agents required to meet the target performance constraints with 95% confidence
def find_optimal_staffing():
    num_calls = ARRIVALS_PER_MINUTE * 60 * HOURS_OPEN
    num_agents = int(np.ceil(LAMBDA / MU))  # Start with the theoretical minimum
    confidence_interval = 1.96  # For 95% confidence
    optimal_found = False
    
    while not optimal_found:
        average_wait_time, abandonment_rate = simulate_call_center(num_agents, num_calls)
        # Check if both performance constraints are met within the 95% confidence interval
        if average_wait_time <= MAX_WAIT_TIME and abandonment_rate <= MAX_ABANDONMENT_RATE:
            # Check the upper bound of the confidence interval for abandonment rate
            if abandonment_rate + confidence_interval * np.sqrt((abandonment_rate * (1 - abandonment_rate)) / num_calls) <= MAX_ABANDONMENT_RATE:
                optimal_found = True
            else:
                num_agents += 1  # Increase agents if upper bound exceeds the constraint
        else:
            num_agents += 1  # Increase agents if constraints are not met
    return num_agents

# Calculate the optimal number of agents needed
optimal_agents = find_optimal_staffing()
optimal_agents


In [ ]:
# Plotting the number of calls in progress over time assuming instant call answering

# Simulate call arrivals
arrival_times = np.cumsum(np.random.exponential(scale=1/LAMBDA, size=int(ARRIVALS_PER_MINUTE * 60 * HOURS_OPEN)))

# Assuming calls are answered instantly, call end times are just the arrival times plus service time
call_end_times = arrival_times + MEAN_SERVICE_TIME

# Time points to evaluate the number of calls in progress
time_points = np.arange(0, SECONDS_OPEN + 1, 1)
num_calls_in_progress = np.zeros_like(time_points)

# Count the number of calls in progress at each time point
for i, time_point in enumerate(time_points):
    num_calls_in_progress[i] = np.sum((arrival_times <= time_point) & (call_end_times > time_point))

# Plotting
fig = go.Figure(data=[
    go.Scatter(x=time_points, y=num_calls_in_progress, mode='lines', name='Calls in Progress')
])

# Update the layout
fig.update_layout(
    title="Number of Calls in Progress Over Time",
    xaxis_title="Time (seconds)",
    yaxis_title="Number of Calls in Progress",
    xaxis=dict(range=[0, SECONDS_OPEN]),
    yaxis=dict(range=[0, num_calls_in_progress.max() + 1])
)

# Show the figure
fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=num_calls_in_progress,
    histnorm='probability density',
    name='Call Distribution',
    nbinsx=60,  # change accordingly
    marker_color='lightblue',
    opacity=0.75
))

fig.add_trace(go.Scatter(
    x=x_values,
    y=pdf,
    name='Normal Distribution',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Normal Distribution Fit to Number of People on Call',
    xaxis_title='Number of People on Call',
    yaxis_title='Probability Density',
    bargap=0.2,  
)

fig.show()

In [ ]:
average_calls_in_progress = num_calls_in_progress.mean()
print(average_calls_in_progress)

peak_max = num_calls_in_progress.max()
print(peak_max)

percentiles = np.percentile(num_calls_in_progress, [25, 50, 75, 90, 95])
print(percentiles)